In [1]:
import re
from queue import Queue
from typing import Iterable
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ete3
from ete3 import PhyloTree
from scipy.stats import entropy

In [2]:
path_to_tree = "../data/interim/iqtree_runs/drun1/anc.treefile"
path_to_states = "../data/interim/iqtree_runs/drun1/anc.state"
path_to_leaves = "../data/interim/leaves_states.tsv"

In [3]:
mutations = pd.read_csv("../data/processed/nematoda_mutations.csv")
edge_mutspec_all = pd.read_csv("../data/processed/nematoda_mutspec_all.csv")
edge_mutspec_syn = pd.read_csv("../data/processed/nematoda_mutspec_syn.csv")
edge_mutspec_ff = pd.read_csv("../data/processed/nematoda_mutspec_ff.csv")

In [4]:
tree = tree = PhyloTree(path_to_tree, format=1)

anc = pd.read_csv(path_to_states, sep="\t", comment='#',)
leaves = pd.read_csv(path_to_leaves, sep="\t")

states = pd.concat([anc, leaves]).sort_values(["Node", "Part", "Site"])
print(states.shape)
states.head()

(2866578, 8)


,Node,Part,Site,State,p_A,p_C,p_G,p_T
6516,Acanthocheilonema_viteae,1,1,A,1.0,0.0,0.0,0.0
6517,Acanthocheilonema_viteae,1,2,T,0.0,0.0,0.0,1.0
6518,Acanthocheilonema_viteae,1,3,T,0.0,0.0,0.0,1.0
6519,Acanthocheilonema_viteae,1,4,T,0.0,0.0,0.0,1.0
6520,Acanthocheilonema_viteae,1,5,T,0.0,0.0,0.0,1.0


In [8]:
edge_mutspec_ff

,Mut,ObsFr,RefNuc,AltNuc,Divisor,RawMutSpec,MutSpec,RefNode,AltNode
0,T>A,162,T,A,5361,0.030218,0.522623,Node1,Philometroides_sanguineus
1,T>G,75,T,G,5361,0.013990,0.241955,Node1,Philometroides_sanguineus
2,T>C,30,T,C,5361,0.005596,0.096782,Node1,Philometroides_sanguineus
3,A>G,8,A,G,1953,0.004096,0.070845,Node1,Philometroides_sanguineus
4,G>A,4,G,A,1678,0.002384,0.041228,Node1,Philometroides_sanguineus
...,...,...,...,...,...,...,...,...,...
3391,G>T,7,G,T,1763,0.003971,0.057672,Node94,Cylicocyclus_ashworthi
3392,A>C,2,A,C,2775,0.000721,0.010469,Node94,Cylicocyclus_ashworthi
3393,G>C,2,G,C,1763,0.001134,0.016478,Node94,Cylicocyclus_ashworthi
3394,C>A,1,C,A,760,0.001316,0.019112,Node94,Cylicocyclus_ashworthi


In [5]:
def prepare_mut_share_file(mutspec: pd.DataFrame, filename: str = None, sbs1="C>T", sbs2=None):
    assert bool(re.match("[ACGT]>[ACGT]", sbs1)), "sbs1 is not appropriate"
    if isinstance(sbs2, str):
        assert bool(re.match("[ACGT]>[ACGT]", sbs2)), "sbs2 is not appropriate"

    cols = ["RefNode", "AltNode", "MutSpec"]
    mcol = cols[-1]

    for c in cols:
        assert c in mutspec, f"Column {c} is not in mutspec dataframe"
    
    shares = mutspec[mutspec.Mut == sbs1][cols]
    if sbs2 is not None:
        denominator = mutspec[mutspec.Mut == sbs2][mcol]
        shares[mcol] = shares[mcol] / denominator.values
    if isinstance(filename, str):
        shares.to_csv(filename, "\t", index=None)
    return shares

In [9]:
for n1 in "ACGT":
    for n2 in "ACGT":
        if n1 == n2:
            continue

        sbs = f"{n1}>{n2}"
        prepare_mut_share_file(edge_mutspec_ff, f"../data/processed/sbs_on_tree/{sbs}_ff.tsv", sbs)

In [13]:
prepare_mut_share_file(edge_mutspec_ff, "../data/processed/sbs_on_tree/A>T_T>A_ratio_ff.tsv", sbs1="A>T", sbs2="T>A")

,RefNode,AltNode,MutSpec
5,Node1,Philometroides_sanguineus,0.050833
15,Node1,Dracunculus_medinensis,1.478081
24,Node1,Node2,5.490015
37,Node2,Node3,1.313317
52,Node2,Camallanus_cotti,0.172238
...,...,...,...
3336,Node92,Cylicocyclus_insigne,2.627716
3350,Node93,Node94,1.127746
3361,Node93,Cylicocyclus_nassatus,1.914200
3376,Node94,Cylicocyclus_radiatus,1.242176


### Prepare files for trees with T-shares

In [ ]:
# all sites
one_genome = states[states.Node == states.Node.values[0]].reset_index(drop=True)
aln_len = one_genome.shape[0]
share_of_T = states.groupby("Node").p_T.sum() / aln_len
share_of_T = share_of_T.reset_index().reset_index()
share_of_T.columns = ["Nothing", "Node", "PrT"]
share_of_T.to_csv("../data/processed/percent_of_T.tsv", sep="\t", index=None)

In [ ]:
# fourfold sites
ff_states = ...

import sys

sys.path.append("/home/kpotoh/birds/ff_codonsscripts")

from utils import extract_ff_codons


ff_codons = extract_ff_codons(5)
print(len(ff_codons), ff_codons)

36 {'CGG', 'ACC', 'TCA', 'CCG', 'GGG', 'AGA', 'ACT', 'TCT', 'AGG', 'CGC', 'GTA', 'GCA', 'GTC', 'ACG', 'GGA', 'GGT', 'TCG', 'CTG', 'GCT', 'GTT', 'CTT', 'CCC', 'GCC', 'ACA', 'CCA', 'CTC', 'CTA', 'GGC', 'AGC', 'CGT', 'AGT', 'TCC', 'GCG', 'CGA', 'CCT', 'GTG'}


In [ ]:
codons = []
codon = ""
for i, st in enumerate(states.State.values, 1):
    codon += st
    if i % 3 == 0:
        for _ in range(3):
            codons.append(codon)
        codon = ""

In [ ]:
states["Codon"] = codons
states["NucInCodon"] = (states.Site - 1) % 3 + 1

In [ ]:
ff_states = states[states.NucInCodon == 3]
ff_states = ff_states[ff_states.Codon.isin(ff_codons)]
ff_states.shape

(330147, 10)

In [ ]:
one_genome_ff = ff_states[ff_states.Node == ff_states.Node.values[0]].reset_index(drop=True)
aln_len_ff = one_genome_ff.shape[0]

share_of_T_ff = ff_states.groupby("Node").p_T.sum() / aln_len_ff
share_of_T_ff = share_of_T_ff.reset_index().reset_index()
share_of_T_ff.columns = ["Nothing", "Node", "PrT"]
share_of_T_ff.to_csv("../data/processed/percent_of_T_ff.tsv", sep="\t", index=None)